In [2]:
!pip3 install -qU llama-index llama-cpp-python fpdf

^C
ERROR: Operation cancelled by user

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python3.11 -m pip install --upgrade pip


In [3]:

from llama_index.llms import LlamaCPP
from llama_index.llms.llama_utils import (
    messages_to_prompt,
    completion_to_prompt,
)

from fpdf import FPDF

In [9]:
model_url = 'llama-2-7b-chat.Q3_K_M.gguf'

In [12]:
# LOAD MODEL 
llm = LlamaCPP(
    # You can pass in the URL to a GGML model to download it automatically
    model_path=model_url,
    # optionally, you can set the path to a pre-downloaded model instead of model_url
    # model_url=None,
    temperature=0.1,
    max_new_tokens=256,
    # llama2 has a context window of 4096 tokens, but we set it lower to allow for some wiggle room
    context_window=4096,
    # kwargs to pass to __call__()
    generate_kwargs={},
    # kwargs to pass to __init__()
    # set to at least 1 to use GPU
    # model_kwargs={"n_gpu_layers": 1},
    # transform inputs into Llama2 format
    messages_to_prompt=messages_to_prompt,
    completion_to_prompt=completion_to_prompt,
    verbose=True,
)

# llm = LlamaCPP(
#         model_url="https://huggingface.co/TheBloke/Llama-2-13B-chat-GGUF/resolve/main/llama-2-13b-chat.Q3_K_M.gguf",
#         model_path=None,
#         temperature=0,
#         max_new_tokens=512,
#         context_window=10000,
#         generate_kwargs={},
#         verbose=True,
#     )


llama_model_loader: loaded meta data with 19 key-value pairs and 291 tensors from llama-2-7b-chat.Q3_K_M.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = LLaMA v2
llama_model_loader: - kv   2:                       llama.context_length u32              = 4096
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 11008
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - kv   7:                 llama.attention.head_count u32              = 

In [5]:
prompt ='''
Write a coverletter for the user with full sentences. 
Use their work history and job details such as the job title and keywords provided below:

job title: {}
work experience: {}
keywords: {}
'''


In [6]:
basic_experience = '''
    As a Junior Developer at Norconsult, I've played a key role in enhancing our business intelligence app, implementing diverse visualizations using CraftJS, ChartJS, Leaflet, and React libraries. I improved data accessibility by designing a system to access various sources and storing data in PostgreSQL. Strengthening data analysis capabilities, I utilized CubeJS for querying, measures, and dimensions. Additionally, I developed a robust REST API using Node.js, Express, middleware, and Winston to enhance application functionality and reliability. I also contributed to AI and machine learning advancements through research and prototyping.

    My recent project, the Heads-Up Display Smart Helmet, received the "Best Safety Analysis" award, showcasing my innovation. I developed and tested it using Raspberry Pi 4 and Pi Camera for real-time video stream processing. Implementing object detection using TensorFlow lite models, I optimized performance with a transition from Python to C++. Demonstrating expertise in embedded systems design, I addressed task management through multiprocessing and multithreading.
'''
title=input("what is the position you are applying for?")
work= basic_experience #input("What are some of you notable work experiences?")
jobkeys= input("What are some of job description keywords")

In [7]:
final_question = prompt.join((title, work, jobkeys))

In [13]:
response_iter = llm.stream_complete(final_question)
str_res = ""
for response in response_iter:
    str_res += response.delta
    print(response.delta, end="", flush=True)

  Dear Hiring Manager,
I am writing to express my strong interest in the Web Developer position at your esteemed organization. As a skilled and dedicated web developer with a passion for creating innovative solutions, I am confident that I would be an excellent fit for this role.
With over two years of experience as a Junior Developer at Norconsult, I have honed my skills in web development, including proficiency in JavaScript, React, and various other technologies. My work history includes developing business intelligence apps using CraftJS, ChartJS, Leaflet, and React libraries, as well as designing a system to access data from multiple sources and storing it in PostgreSQL. I have also strengthened data analysis capabilities by utilizing CubeJS for

In [8]:
import textwrap

def text_to_pdf(text, filename):
    a4_width_mm = 210
    pt_to_mm = 0.35
    fontsize_pt = 10
    fontsize_mm = fontsize_pt * pt_to_mm
    margin_bottom_mm = 10
    character_width_mm = 7 * pt_to_mm
    width_text = a4_width_mm / character_width_mm

    pdf = FPDF(orientation='P', unit='mm', format='A4')
    pdf.set_auto_page_break(True, margin=margin_bottom_mm)
    pdf.add_page()
    pdf.set_font(family='Arial', size=fontsize_pt)
    splitted = text.split('\n')

    for line in splitted:
        lines = textwrap.wrap(line, width_text)

        if len(lines) == 0:
            pdf.ln()

        for wrap in lines:
            pdf.cell(0, fontsize_mm, wrap, ln=1)

    pdf.output(filename, 'F')

In [9]:
print(str_res)
# save FPDF() class into a 
# variable pdf
pdf = FPDF()

text_to_pdf(str_res, "coverletter.pdf")

  Dear Hiring Manager,
I am writing to express my interest in the Software Developer position at RPYX, and I am excited to submit my application. As a skilled and experienced developer, I am confident that I can bring value to your team and contribute to the success of your projects.
In my current role as a Junior Developer at Norconsult, I have honed my skills in full stack development, including architecting, designing, crafting, and maintaining software applications. I have worked extensively with SQL and REST APIs, and I am proficient in using various libraries such as CraftJS, ChartJS, Leaflet, and React to implement diverse visualizations. My experience has also allowed me to strengthen data analysis capabilities by utilizing CubeJS for querying, measures, and dimensions.
One of my most notable projects is the Heads-Up Display Smart Helmet, which I developed and tested using Raspberry Pi 4 and Pi Camera for real-time video stream processing. I implemented object detection using T